In [1]:
pip install transformers datasets evaluate accelerate pillow torchvision scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 899.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from kaggle_secrets import UserSecretsClient
huggingface_token = UserSecretsClient().get_secret("huggingface_token")

from huggingface_hub import login

login(token=huggingface_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**Load dataset**

In [3]:
from datasets import load_dataset

datasets = load_dataset("vishnun0027/Crop_Disease", split="train")

Generating train split:   0%|          | 0/3747 [00:00<?, ? examples/s]

In [4]:
datasets

Dataset({
    features: ['image', 'label'],
    num_rows: 3747
})

In [5]:
datasets = datasets.train_test_split(test_size=0.2)


In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 2997
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 750
    })
})

In [7]:
datasets["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=800x800>,
 'label': 16}

In [8]:
labels = datasets["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [9]:
id2label[str(5)]

'Botrytis'

**Preprocess**

In [10]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

2024-06-24 06:31:48.090448: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 06:31:48.090559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 06:31:48.229612: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

In [11]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [12]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [13]:
datasets = datasets.with_transform(transforms)

In [14]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

**Evaluate**

In [15]:
import evaluate

accuracy = evaluate.load("accuracy")

In [16]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

**Train**

In [17]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir="Crop_Disease_model_1",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=18,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=datasets["train"],
    eval_dataset=datasets["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,2.974000,2.928837,0.157333
2,2.399500,2.322290,0.570667
4,1.896500,1.837725,0.613333
6,1.494200,1.580719,0.669333
8,1.297500,1.460502,0.661333
10,1.145700,1.362178,0.692000
12,1.029600,1.296803,0.713333
14,0.931700,1.262506,0.705333
16,0.914000,1.259339,0.701333
17,0.901300,1.248250,0.700000


/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to 

TrainOutput(global_step=414, training_loss=1.528570953198677, metrics={'train_runtime': 2802.3498, 'train_samples_per_second': 19.25, 'train_steps_per_second': 0.148, 'total_flos': 4.0926246613036646e+18, 'train_loss': 1.528570953198677, 'epoch': 17.617021276595743})

In [19]:
trainer.push_to_hub()


CommitInfo(commit_url='https://huggingface.co/vishnun0027/Crop_Disease_model_1/commit/d4d850fe42708967e26547f86a877d8b72389d6d', commit_message='End of training', commit_description='', oid='d4d850fe42708967e26547f86a877d8b72389d6d', pr_url=None, pr_revision=None, pr_num=None)